*italicized text*# Exploratory Analysis
[link text](https://)

# Explore

In [ ]:
#import libraries and view dataset
import numpy as np
import pandas as pd
from pandas import Series
from numpy.random import randn
import csv #csv library col-name
import scipy.io

#import data from google drive
from google.colab import drive
drive.mount('/content/drive')

# Load the metadata file
metadata = pd.read_csv('/content/drive/MyDrive/BatteryProjectData/cleaned_dataset/metadata.csv')

# Filter for battery B0005
battery_data = metadata[metadata['battery_id'] == 'B0005']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# separate charge cycles
charge_cycles = battery_data[battery_data['type'] == 'charge']

# Function to process individual cycle files
def process_charge_cycle_files(cycles_df):
    all_cycles = []
    cycle_num = 1
    for index, row in cycles_df.iterrows():
        cycle_data = pd.read_csv(f'/content/drive/MyDrive/BatteryProjectData/cleaned_dataset/data/{row["filename"]}')
        cycle_data['cycle_number'] = cycle_num
        cycle_data['start_time'] = row['start_time']
        cycle_data['ambient_temperature'] = row['ambient_temperature']
        all_cycles.append(cycle_data)
        cycle_num += 1
    return pd.concat(all_cycles)

# Process charge and discharge cycles
charge_data = process_charge_cycle_files(charge_cycles)

try:
    # Save Charge and discharge data as csv files
    charge_data.to_csv('/content/drive/MyDrive/BatteryProjectData/MyCleanedData/charge_data_B0005.csv', index=False)


    print(f"charge_data Saved to {'charge_data_B0005'}")

except Exception as e:
    print(f"Could not save charge_data: {e}")

charge_data Saved to charge_data_B0005


In [ ]:
# separate discharge cycles
discharge_cycles = battery_data[battery_data['type'] == 'discharge']

# Function to process individual cycle files
def process_discharge_cycle_files(cycles_df):
    all_cycles = []
    cycle_num = 1
    for index, row in cycles_df.iterrows():
        cycle_data = pd.read_csv(f'/content/drive/MyDrive/BatteryProjectData/cleaned_dataset/data/{row["filename"]}')
        cycle_data['cycle_number'] = cycle_num
        cycle_data['start_time'] = row['start_time']
        cycle_data['ambient_temperature'] = row['ambient_temperature']
        cycle_data['Capacity'] = row['Capacity']
        all_cycles.append(cycle_data)
        cycle_num += 1
    return pd.concat(all_cycles)

# Process charge and discharge cycles
discharge_data = process_discharge_cycle_files(discharge_cycles)

try:
    # Save Charge and discharge data as csv files
    discharge_data.to_csv('/content/drive/MyDrive/BatteryProjectData/MyCleanedData/discharge_data_B0005.csv', index=False)

    print(f"discharge_data Saved to {'discharge_data_B0005'}")

except Exception as e:
    print(f"Could not save discharge_data: {e}")

discharge_data Saved to discharge_data_B0005


In [ ]:
# separate impedance cycles
impedance_cycles = battery_data[battery_data['type'] == 'impedance']

# Function to process individual cycle files
def process_impedance_cycle_files(cycles_df):
    all_cycles = []
    cycle_num = 1
    for index, row in cycles_df.iterrows():
        cycle_data = pd.read_csv(f'/content/drive/MyDrive/BatteryProjectData/cleaned_dataset/data/{row["filename"]}')
        cycle_data['cycle_number'] = cycle_num
        cycle_data['start_time'] = row['start_time']
        cycle_data['ambient_temperature'] = row['ambient_temperature']
        cycle_data['Re'] = row['Re']
        cycle_data['Rct'] = row['Rct']
        all_cycles.append(cycle_data)
        cycle_num += 1
    return pd.concat(all_cycles)

# Process charge and discharge cycles
impedance_data = process_impedance_cycle_files(impedance_cycles)

# Function to process the complex imaginary numbers
def split_complex_number(complex_str):
    try:
        # Convert string to complex number
        complex_num = complex(complex_str.replace('j', 'j '))

        # Return real and imaginary parts
        return complex_num.real, complex_num.imag
    except ValueError:
        # Handle cases where conversion fails
        return None, None

# Columns to process
columns_to_process = ['Sense_current', 'Battery_current', 'Current_ratio', 'Battery_impedance']

for column in columns_to_process:
    # Split each column into real and imaginary parts
    impedance_data[column + '_real'], impedance_data[column + '_imag'] = zip(*impedance_data[column].apply(split_complex_number))

# drop the original columns if they are no longer needed
impedance_data = impedance_data.drop(columns=columns_to_process)

try:
    # Save Charge and discharge data as csv files
    impedance_data.to_csv('/content/drive/MyDrive/BatteryProjectData/MyCleanedData/impedance_data_B0005.csv', index=False)

    print(f"impedance_data Saved to {'impedance_data_B0005'}")

except Exception as e:
    print(f"Could not save impedance_data: {e}")

impedance_data Saved to impedance_data_B0005


In [ ]:
# combine all experimental cycles for B0005 together

# Function to process a single cycle file
def process_cycle(row):
    cycle_data = pd.read_csv(f'/content/drive/MyDrive/BatteryProjectData/cleaned_dataset/data/{row["filename"]}')
    cycle_data['cycle_type'] = row['type']
    cycle_data['start_time'] = row['start_time']
    cycle_data['ambient_temperature'] = row['ambient_temperature']

    # Additional attributes for discharge and impedance cycles
    if row['type'] == 'discharge':
        cycle_data['Capacity'] = row['Capacity']
    elif row['type'] == 'impedance':
        cycle_data['Re'] = row['Re']
        cycle_data['Rct'] = row['Rct']

    return cycle_data

# Process all cycles in the order they appear in the metadata
all_cycles = [process_cycle(row) for _, row in battery_data.iterrows()]
all_data = pd.concat(all_cycles)

try:
    # Save merged data as a csv file
    all_data.to_csv('/content/drive/MyDrive/BatteryProjectData/MyCleanedData/all_data.csv', index=False)
    print("Data Saved to 'all_data.csv'")
except Exception as e:
    print(f"Could not save data: {e}")


Data Saved to 'all_data.csv'
